In [1]:
## import libraries
import requests
import pandas as pd
import string
import pickle
import time
# pip3 install neo4j-driver
from py2neo import Graph, data

In [2]:
# %%
## set up the neo4j connection
graph = Graph("bolt://localhost:7687", auth=("neo4j", "rossisaloser"))


In [3]:
"""
Types of entities that are needed:
- JOB Title
- Technology Tool
- Core Competency
    - Level of skill required for a certain JOB
- Specialist Task
    - Percentage of time required for the skill in the JOB
- 
"""

'\nTypes of entities that are needed:\n- JOB Title\n- Technology Tool\n- Core Competency\n    - Level of skill required for a certain JOB\n- Specialist Task\n    - Percentage of time required for the skill in the JOB\n- \n'

Occupations

In [4]:
occ_df = pd.read_csv('./process_data/occupations.csv')

In [5]:
occ_df.head()

,ANZSCO_Code,ANZSCO_Title,ANZSCO_Desc
0,1111,Chief Executives and Managing Directors,Chief Executives and Managing Directors determ...
1,1112,General Managers,"General Managers plan, organise, direct, contr..."
2,1213,Livestock Farmers,"Livestock Farmers plan, organise, control, coo..."
3,1214,Mixed Crop and Livestock Farmers,"Mixed Crop and Livestock Farmers plan, organis..."
4,1321,Corporate Services Managers,Corporate Services Managers organise and direc...


Competencies and Occupations at same time

In [6]:
comps_df = pd.read_csv('./process_data/core_competencies.csv')

In [7]:
comps_df.head()

,ANZSCO_Code,ANZSCO_Title,Core_Competencies,Score,Proficiency_level,Anchor_value
0,1111,Chief Executives and Managing Directors,Numeracy,6,Intermediate,Calculate the square footage of a new home und...
1,1111,Chief Executives and Managing Directors,Digital engagement,6,Intermediate,Use software on a portable device to document ...
2,1111,Chief Executives and Managing Directors,Teamwork,7,Intermediate,"Share knowledge, experience, information and r..."
3,1111,Chief Executives and Managing Directors,Writing,7,Intermediate,Write a detailed literature review
4,1111,Chief Executives and Managing Directors,Reading,8,High,Read and analyse information comparing the imp...


In [8]:
def occDfRowToNeo(row, pointer):

    ANZSCO_Code = row['ANZSCO_Code']
    ANZSCO_Title = row['ANZSCO_Title']
    Core_Competencies = row['Core_Competencies']
    Score = row['Score']
    Proficiency_level = row['Proficiency_level']    

    occ_query = """
    MERGE (o:Occupation {ANZSCO_Code: $anzsco_code})
    ON CREATE SET o.ANZSCO_Title = $anzsco_title,
                o.ANZSCO_Code = $anzsco_code
    MERGE (c:Competency {Core_Competencies: $comp_name})
    ON CREATE SET c.Core_Competencies = $comp_name 
    WITH o,c
    MERGE (o)-[:REQUIRES_COMPETENCY {Score: $score, Proficiency: $prof}]->(c)         
    """

    pointer.run(
        occ_query,
        anzsco_title=ANZSCO_Title,
        anzsco_code=ANZSCO_Code,
        comp_name=Core_Competencies,
        score=Score,
        prof=Proficiency_level
        )

In [9]:
for index, row in comps_df.iterrows():
    occDfRowToNeo(row,graph)

Tasks and their clusters,families

In [10]:
tasks_df = pd.read_csv('./process_data/Specialist_Tasks.csv')

In [11]:
tasks_df.head()

,ANZSCO_Code,ANZSCO_Title,Specialist_Task,% of time spent on task,Specialist_Cluster,% of time spent on cluster,Cluster_Family,% of time spent on family
0,1111,Chief Executives and Managing Directors,Direct financial operations,16%,"Manage, monitor and undertake financial activi...",16%,Business operations and financial activities,52%
1,1111,Chief Executives and Managing Directors,"Direct organisational operations, projects or ...",13%,"Manage services, staff or activities",26%,Business operations and financial activities,52%
2,1111,Chief Executives and Managing Directors,"Direct sales, marketing or customer service ac...",10%,"Manage services, staff or activities",26%,Business operations and financial activities,52%
3,1111,Chief Executives and Managing Directors,Confer with peers to accomplish work,8%,Communicate and collaborate,9%,Communication and collaboration,11%
4,1111,Chief Executives and Managing Directors,Analyse data to assess operational or project ...,8%,Use data to inform operational decisions,12%,"Data, analytics, and databases",12%


In [12]:
tasks_df['task_percent'] = tasks_df['% of time spent on task'].str.replace('%','')

In [13]:
tasks_df['task_percent'] = pd.to_numeric(tasks_df['task_percent'])

In [14]:
def taskDfRowToNeo(row, pointer):
    
    ANZSCO_Code = row['ANZSCO_Code']
    ANZSCO_Title = row['ANZSCO_Title']
    Specialist_Task = row['Specialist_Task']
    task_percent = row['task_percent']
    Specialist_Cluster = row['Specialist_Cluster']
    Cluster_Family = row['Cluster_Family']    

    tsk_query = """
    MATCH (o:Occupation {ANZSCO_Code: $anzsco_code})
    MERGE (t:Task {Task_Name: $tsk})
        ON CREATE SET t.Task_Name = $tsk 
    WITH o,t
    MERGE (o)-[:INVOLVES_TASK {task_percent: $tsk_prct}]->(t)         
    """

    pointer.run(
        tsk_query,
        tsk=Specialist_Task,
        anzsco_code=ANZSCO_Code,
        tsk_prct=task_percent
        )

    cluster_query = """
    MATCH (t:Task {Task_Name: $tsk})
    MERGE (c:Cluster {Cluster_Name: $cluster})
        ON CREATE SET c.Cluster_Name = $cluster 
    WITH t,c
    MERGE (t)-[:BELONGS_TO_CLUSTER]->(c)         
    """

    pointer.run(
        cluster_query,
        tsk=Specialist_Task,
        cluster=Specialist_Cluster
        )

    family_query = """
    MATCH (c:Cluster {Cluster_Name: $cluster})
    MERGE (f:Family {Family_Name: $family})
        ON CREATE SET f.Family_Name = $family 
    WITH c,f
    MERGE (c)-[:BELONGS_TO_FAMILY]->(f)         
    """

    pointer.run(
        family_query,
        cluster=Specialist_Cluster,
        family=Cluster_Family
        )

    print(f'occupation: { ANZSCO_Title }, task: {Specialist_Task}')

In [15]:
for index, row in tasks_df.iterrows():
    taskDfRowToNeo(row,graph)

occupation: Chief Executives and Managing Directors, task: Direct financial operations
occupation: Chief Executives and Managing Directors, task: Direct organisational operations, projects or services
occupation: Chief Executives and Managing Directors, task: Direct sales, marketing or customer service activities
occupation: Chief Executives and Managing Directors, task: Confer with peers to accomplish work
occupation: Chief Executives and Managing Directors, task: Analyse data to assess operational or project effectiveness
occupation: Chief Executives and Managing Directors, task: Manage human resources activities
occupation: Chief Executives and Managing Directors, task: Analyse data to inform operational decisions or activities
occupation: Chief Executives and Managing Directors, task: Communicate organisational policies and procedures
occupation: Chief Executives and Managing Directors, task: Manage construction activities
occupation: Chief Executives and Managing Directors, task: 

In [17]:
tools_df = pd.read_csv('./process_data/technology_tools.csv')

In [18]:
tools_df.head()

,ANZSCO_Code,ANZSCO_Title,Technology_tool
0,1111,Chief Executives and Managing Directors,Enterprise resource planning ERP software
1,1111,Chief Executives and Managing Directors,Project management software
2,1111,Chief Executives and Managing Directors,Accounting software
3,1111,Chief Executives and Managing Directors,Data base user interface and query software
4,1112,General Managers,Enterprise resource planning ERP software


In [ ]:
def toolsDfRowToNeo(row, pointer):
    
    ANZSCO_Code = row['ANZSCO_Code']
    ANZSCO_Title = row['ANZSCO_Title']
    Technology_tool = row['Technology_tool']

    tool_query = """
    MATCH (o:Occupation {ANZSCO_Code: $anzsco_code})
    MERGE (t:Tool {Tool_Name: $tool})
        ON CREATE SET t.Tool_Name = $tool 
    WITH o,t
    MERGE (o)-[:UTILISES_TOOL]->(t)         
    """

    pointer.run(
        tool_query,
        tool=Technology_tool,
        anzsco_code=ANZSCO_Code
        )

In [ ]:
com_deet_df = pd.read_csv('/process_data/Core_competency_details.csv')